In [1]:
import math
from tensorboardX import SummaryWriter
sess = None

In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
import gym
env = gym.make('CartPole-v1')
observation_shape = env.observation_space.shape
n_actions = env.action_space.n
shape =(None, ) + observation_shape
print(shape)

(None, 4)


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
# The typical imports
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Imports specifically so we can render outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display


def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    #plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    display(display_animation(anim, default_mode='loop'))

In [5]:
class EnvCreator(object):
    def __init__(self, name):
        self.name = name
    def __call__(self, ):
        return gym.make(self.name)

In [6]:
from a2c import A2CAgent
import tr_helpers
import networks
import ray
ray.init()
a2c_cartpole_config = {
    'GAMMA' : 0.99,
    'TAU' : 0.9,
    'LEARNING_RATE' : 1e-3,
    'NAME' : 'A2C',
    'SCORE_TO_WIN' : 200,
    'EPISODES_TO_LOG' : 20, 
    'LIVES_REWARD' : 5,
    'GRAD_NORM' : 0.5,
    'ENTROPY_COEF' : 0.001,
    'NUM_ACTORS' : 1,
    'STEPS_NUM' : 100,
    'TRUNCATE_GRADS' : False,
    'ENV_NAME' : 'CartPole-v1',
    'PPO' : True,
    'E_CLIP' : 0.2
}
#self, sess, env_name, observation_shape, actions_num, config = default_config
observation_shape = env.observation_space.shape
actions_num = env.action_space.n
action_shape = env.action_space.shape
print(action_shape)
agent = A2CAgent(sess,'cartpole', observation_shape, actions_num, action_shape, a2c_cartpole_config)
agent.train()

2019-04-19 23:07:41,985	INFO node.py:278 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-19_23-07-41_8957/logs.
2019-04-19 23:07:42,094	INFO services.py:396 -- Waiting for redis server at 127.0.0.1:20103 to respond...
2019-04-19 23:07:42,228	INFO services.py:396 -- Waiting for redis server at 127.0.0.1:54977 to respond...
2019-04-19 23:07:42,234	INFO services.py:798 -- Starting Redis shard with 10.0 GB max memory.
2019-04-19 23:07:42,284	INFO services.py:1360 -- Starting the Plasma object store with 13.477717606 GB memory using /dev/shm.



View the web UI at http://localhost:8889/notebooks/ray_ui.ipynb?token=2722988c1756a2f1cd6a2e7d9f191cada5d548329e8efc12

()
a_loss -5.715195
c_loss 31.932016
entropy 0.6800946
a_loss -6.3650527
c_loss 35.11039
entropy 0.67406386
a_loss -5.4768553
c_loss 20.50103
entropy 0.6651118
a_loss -5.913965
c_loss 24.676975
entropy 0.65962
a_loss -6.091332
c_loss 19.431816
entropy 0.6678729
a_loss -5.9127703
c_loss 15.6322975
entropy 0.6680387
a_loss -6.3580647
c_loss 14.666348
entropy 0.6706948
a_loss -6.3919773
c_loss 12.68245
entropy 0.6718155
a_loss -6.2936163
c_loss 11.4663
entropy 0.67645365
a_loss -6.6172905
c_loss 9.121059
entropy 0.6559523
a_loss -6.701978
c_loss 7.2668934
entropy 0.66992533
a_loss -6.353337
c_loss 9.626527
entropy 0.68512815
a_loss -7.1947136
c_loss 5.919004
entropy 0.68709564
a_loss -5.8204055
c_loss 5.6672707
entropy 0.6857248
a_loss -6.871364
c_loss 5.332251
entropy 0.68995607
a_loss -7.0385227
c_loss 5.8248835
entropy 0.6886406
a_loss -8.344022
c_loss 6.4070764
entr

a_loss -8.341395
c_loss 2.8380206
entropy 0.6439216
a_loss -8.176266
c_loss 3.6359594
entropy 0.6234594
a_loss -7.7236066
c_loss 1.9640031
entropy 0.6289551
a_loss -7.2072377
c_loss 2.5484207
entropy 0.6600255
a_loss -7.5394897
c_loss 1.2751739
entropy 0.6655485
a_loss -7.0060973
c_loss 1.6861588
entropy 0.65574455
a_loss -7.117075
c_loss 1.0057638
entropy 0.6612391
a_loss -7.0293665
c_loss 2.348419
entropy 0.6715584
a_loss -6.8609633
c_loss 3.8632886
entropy 0.6057905
a_loss -7.756128
c_loss 4.101833
entropy 0.65329516
a_loss -6.708407
c_loss 1.7534595
entropy 0.6379201
a_loss -8.31552
c_loss 3.6385326
entropy 0.66519946
a_loss -7.2000594
c_loss 1.5662565
entropy 0.6407482
a_loss -7.7367773
c_loss 2.3311043
entropy 0.6105884
a_loss -6.473631
c_loss 2.4854848
entropy 0.67352617
a_loss -7.0551605
c_loss 1.0324115
entropy 0.65145963
a_loss -6.493343
c_loss 2.447078
entropy 0.6282217
a_loss -9.037568
c_loss 2.259796
entropy 0.6684096
a_loss -6.604245
c_loss 2.2362282
entropy 0.66789687
a_

KeyboardInterrupt: 

In [10]:
observation = env.reset()
cum_reward = 0
frames = []
for t in range(5000):
    # Render into buffer. 
    frames.append(env.render(mode = 'rgb_array'))
    action = agent.get_action(observation, True)
    observation, reward, done, info = env.step(action)
    cum_reward += reward
    if done:
        print(cum_reward)
        break
env.close()
display_frames_as_gif(frames)

500.0


KeyboardInterrupt: 